In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
# directory
CSV_PATH = r"C:\Users\User\Desktop\Mirkwood\segmentations\results.csv"

best result

In [ ]:
# Load data
df = pd.read_csv(CSV_PATH)

# Extract image number
df["ImageNumber"] = df["Image"].apply(lambda x: re.search(r'img(\d+)_', x).group(1))

# Extract channel and algorithm
df["Channel"] = df["Image"].apply(lambda x: re.search(r'img\d+_([A-Za-z]+_[A-Za-z]+)_', x).group(1).upper())
df["Algorithm"] = df["Image"].apply(lambda x: re.search(r'_([A-Za-z]+)\.png$', x).group(1).capitalize())

# Find the row with the best Dice score
best_row = df.loc[df["Dice"].idxmax()]

# Show details
print(f"Best Dice Score Overall: {best_row['Dice']:.4f}")
print(f" - Image: {best_row['Image']} (Image {best_row['ImageNumber']})")
print(f" - Channel: {best_row['Channel']}")
print(f" - Algorithm: {best_row['Algorithm']}")


top 10 DICE values comparison

In [ ]:
# load csv
df = pd.read_csv(CSV_PATH)

# ensure proper sorting
top_dice = df.sort_values(by="Dice", ascending=False).head(10)

# extract image names
top_dice_names = top_dice["Image"].tolist()

# display results
print("Top 10 Dice scores:")
for i, name in enumerate(top_dice_names, 1):
    print(f"{i}. {name} (Dice = {top_dice.iloc[i-1]['Dice']:.4f})")


top 3 DICE values for each image

In [ ]:
SORT_BY = "Dice"

# load data
df = pd.read_csv(CSV_PATH)

# extract image number
def extract_number(filename):
    match = re.search(r'\d+', filename)
    return match.group(0) if match else "unknown"

df["ImageNumber"] = df["Image"].apply(extract_number)

# group and get top 3 per image number
top_per_image = df.sort_values(by=SORT_BY, ascending=False).groupby("ImageNumber").head(3)

# display
for image_num, group in top_per_image.groupby("ImageNumber"):
    print(f"\nTop 3 for image {image_num}:")
    for _, row in group.iterrows():
        print(f" - {row['Image']} | {SORT_BY} = {row[SORT_BY]:.4f}")


In [ ]:
# load and preprocess
df = pd.read_csv(CSV_PATH)

def extract_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group(0)) if match else -1

df["ImageNumber"] = df["Image"].apply(extract_number)

# top 3 segmentations per image
top3 = df.sort_values(by="Dice", ascending=False).groupby("ImageNumber").head(3)

# assign image index (1 to 10)
image_ids = sorted(top3["ImageNumber"].unique())[:10]
image_idx_map = {img: idx+1 for idx, img in enumerate(image_ids)}
top3["GroupID"] = top3["ImageNumber"].map(image_idx_map)

# label to identify unique segmentation method
top3["Label"] = top3["Image"]

# plot
plt.figure(figsize=(16, 8))

bar_width = 0.2
spacing = 0.05

unique_labels = top3["Label"].unique()
colors = plt.cm.tab20.colors

label_offsets = {-1: -bar_width-spacing, 0: 0, 1: bar_width+spacing}

# prepare bar positions
for i, label in enumerate(unique_labels):
    subset = top3[top3["Label"] == label]
    for j, row in subset.iterrows():
        x_base = row["GroupID"]
        pos_in_group = list(top3[top3["GroupID"] == row["GroupID"]]["Label"]).index(row["Label"]) - 1
        x_pos = x_base + pos_in_group * (bar_width + spacing)
        plt.bar(x_pos, row["Dice"], width=bar_width, color=colors[i % len(colors)], label=label)

# avoid duplicate legend labels
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(
    by_label.values(), by_label.keys(),
    title="Segmentation", prop={'size': 8}, title_fontsize='10',
    loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=1  # move legend outside
)
plt.tight_layout(rect=[0, 0, 0.85, 1])  # leave space on right side

plt.xticks(range(1, 11), [str(i) for i in range(1, 11)], fontsize=12)
plt.xlabel("Image Index", fontsize=14)
plt.ylabel("Dice Score", fontsize=14)
plt.title("Top 3 Dice Scores per Image", fontsize=16)
plt.tight_layout()
plt.show()


top 3 DICE values per algorithm

In [ ]:
# load data
df = pd.read_csv(CSV_PATH)
SORT_BY = "Dice"

# extract algorithm from filename
def extract_algorithm(filename):
    match = re.search(r'_(Otsu|Yen|IsoData|Moments|Default)\b', filename, re.IGNORECASE)
    return match.group(1).capitalize() if match else "Unknown"

df["Algorithm"] = df["Image"].apply(extract_algorithm)

# group and get top 3 per algorithm
top_per_algorithm = df.sort_values(by=SORT_BY, ascending=False).groupby("Algorithm").head(3)

# display
for algo, group in top_per_algorithm.groupby("Algorithm"):
    print(f"\nTop 3 for algorithm {algo}:")
    for _, row in group.iterrows():
        print(f" - {row['Image']} | {SORT_BY} = {row[SORT_BY]:.4f}")


In [ ]:
SORT_BY = "Dice"

# load data
df = pd.read_csv(CSV_PATH)

# extract algorithm from filename
def extract_algorithm(filename):
    match = re.search(r'_(Otsu|Yen|IsoData|Moments|Default)\b', filename, re.IGNORECASE)
    return match.group(1).capitalize() if match else "Unknown"

df["Algorithm"] = df["Image"].apply(extract_algorithm)

# group and get top 3 per algorithm
top3 = df.sort_values(by=SORT_BY, ascending=False).groupby("Algorithm").head(3)

# assign image index (1 to 10)
image_ids = sorted(top3["Algorithm"].unique())[:10]
image_idx_map = {img: idx+1 for idx, img in enumerate(image_ids)}
top3["GroupID"] = top3["Algorithm"].map(image_idx_map)

# label to identify unique segmentation method
top3["Label"] = top3["Image"]

# plot
plt.figure(figsize=(16, 8))

bar_width = 0.2
spacing = 0.05

unique_labels = top3["Label"].unique()
colors = plt.cm.tab20.colors

label_offsets = {-1: -bar_width-spacing, 0: 0, 1: bar_width+spacing}

# prepare bar positions
for i, label in enumerate(unique_labels):
    subset = top3[top3["Label"] == label]
    for j, row in subset.iterrows():
        x_base = row["GroupID"]
        pos_in_group = list(top3[top3["GroupID"] == row["GroupID"]]["Label"]).index(row["Label"]) - 1
        x_pos = x_base + pos_in_group * (bar_width + spacing)
        plt.bar(x_pos, row["Dice"], width=bar_width, color=colors[i % len(colors)], label=label)

# avoid duplicate legend labels
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(
    by_label.values(), by_label.keys(),
    title="Segmentation", prop={'size': 8}, title_fontsize='10',
    loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=1  # move legend outside
)
plt.tight_layout(rect=[0, 0, 0.85, 1])  # leave space on right side

plt.xticks(range(1, 6), np.unique(top3["Algorithm"]), fontsize=12)
plt.xlabel("Algorithm", fontsize=14)
plt.ylabel("Dice Score", fontsize=14)
plt.title("Top 3 Dice Scores per Algorithm", fontsize=16)
plt.tight_layout()
plt.show()


mean and std per algorithm and channel

In [ ]:
# load CSV
df = pd.read_csv(CSV_PATH)

# extract algorithm and channel
def extract_info(name):
    # match patterns like: img0123_RGB_R_Otsu.tif
    match = re.search(r'_(\w+_\w+|Gray)_(Otsu|Yen|IsoData|Moments|Default)', name, re.IGNORECASE)
    if match:
        channel = match.group(1).upper()
        algorithm = match.group(2).capitalize()
        return pd.Series([channel, algorithm])
    else:
        return pd.Series(["Unknown", "Unknown"])

df[['Channel', 'Algorithm']] = df['Image'].apply(extract_info)

# metrics to analyze
metrics = ["Dice", "IoU", "Accuracy", "Precision", "Specificity"]

# group by channel & algorithm
summary = df.groupby(['Channel', 'Algorithm'])[metrics].agg(['mean', 'std']).round(4)

# flatten column names
summary.columns = [f"{metric}_{stat}" for metric, stat in summary.columns]

# save to CSV
summary.to_csv(r"C:\Users\User\Desktop\Mirkwood\segmentations\metrics_by_algorithm_and_channel.csv")


In [ ]:
# directory
CSV_PATH = r"C:\Users\User\Desktop\Mirkwood\segmentations\metrics_by_algorithm_and_channel.csv"

In [ ]:
# load CSV
df = pd.read_csv(CSV_PATH)

# ensure proper sorting
top_dice = df.sort_values(by="Dice_mean", ascending=False).head(10)

# combine channel and clgorithm into a single label
top_dice_names = [f"{row['Channel']} | {row['Algorithm']}" for _, row in top_dice.iterrows()]

# display results
print("Top 10 Dice scores:")
for i, name in enumerate(top_dice_names, 1):
    print(f"{i}. {name} (Dice_mean = {top_dice.iloc[i-1]['Dice_mean']:.4f})")


In [ ]:
# load
df = pd.read_csv(CSV_PATH)

# split 'Channel' into separate columns
df[['ColorSpace', 'Channel']] = df['Channel'].str.split('_', n=1, expand=True)

# create a pivot table: rows = channels, columns = algorithms, values = Dice_mean
pivot = df.pivot_table(index=['ColorSpace', 'Channel'], columns='Algorithm', values='Dice_mean')

# plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot, annot=True, cmap='YlGnBu', fmt=".4f", linewidths=0.5, linecolor='gray')
plt.title("Mean Dice Scores by Algorithm and Channel")
plt.xlabel("Algorithm")
plt.ylabel("Channel")
plt.tight_layout()
plt.show()


sample image analysis

In [ ]:
# directory
CSV_PATH = r"C:\Users\User\Desktop\Mirkwood\segmentations\results.csv"
TARGET_IMAGES = {'1', '6', '33'}  # Set of image numbers of interest

# load CSV
df = pd.read_csv(CSV_PATH)

# extract image number from filename
def extract_image_number(filename):
    match = re.search(r'img(\d+)', filename)
    return match.group(1) if match else None

df['ImageNumber'] = df['Image'].apply(extract_image_number)

# filter rows for specific image numbers
filtered_df = df[df['ImageNumber'].isin(TARGET_IMAGES)]

# sort results (optional)
filtered_df = filtered_df.sort_values(by=['ImageNumber', 'Dice'], ascending=[True, False])

# display results
for img_num in sorted(TARGET_IMAGES):
    subset = filtered_df[filtered_df['ImageNumber'] == img_num]
    print(f"\n Results for image {img_num}:")
    for _, row in subset.iterrows():
        print(f" - {row['Image']} | Dice = {row['Dice']:.4f} | IoU = {row['IoU']:.4f}")


In [ ]:
# load data
df = pd.read_csv(CSV_PATH)

# extract image number
df["ImageNumber"] = df["Image"].apply(lambda x: re.search(r'img(\d+)_', x).group(1))

# extract Channel and Algorithm
df["Channel"] = df["Image"].apply(lambda x: re.search(r'img\d+_([A-Za-z]+_[A-Za-z]+)_', x).group(1).upper())
df["Algorithm"] = df["Image"].apply(lambda x: re.search(r'_([A-Za-z]+)\.png$', x).group(1).capitalize())

# function to plot heatmap for a single image
def plot_heatmap_for_image(image_num):
    sub_df = df[df["ImageNumber"] == str(image_num)]
    heatmap_data = sub_df.pivot(index="Channel", columns="Algorithm", values="Dice")

    plt.figure(figsize=(12, 8))
    sns.heatmap(heatmap_data, annot=True, fmt=".4f", cmap="YlGnBu", cbar_kws={'label': 'Dice Score'})
    plt.title(f"Dice Scores by Algorithm and Channel - Image {image_num}")
    plt.xlabel("Algorithm")
    plt.ylabel("Channel")
    plt.tight_layout()
    plt.show()

# generate for specific images
for img_num in [1, 6, 33]:
    plot_heatmap_for_image(img_num)


In [ ]:
# load CSV
df = pd.read_csv(CSV_PATH)

# extract info from filename
def parse_filename(filename):
    match = re.match(r"img(\d+)_([A-Za-z]+_[A-Za-z]+)_([A-Za-z]+)\.png", filename)
    if match:
        image_num = int(match.group(1))
        channel = match.group(2).upper()
        algorithm = match.group(3).capitalize()
        return pd.Series([image_num, channel, algorithm])
    else:
        return pd.Series([None, None, None])

df[['ImageNumber', 'Channel', 'Algorithm']] = df['Image'].apply(parse_filename)

# filter for Image 1 and desired channels
target_image = 1
target_channels = ['GRAY_GRAY', 'RGB_G', 'YCRCB_Y', 'YUV_Y', 'CIELAB_L']
filtered_df = df[
    (df['ImageNumber'] == target_image) &
    (df['Channel'].isin(target_channels))
]

# list metric columns
metric_columns = ['Dice', 'IoU', 'Accuracy', 'Precision', 'Specificity']

# create heatmaps
for metric in metric_columns:
    pivot = filtered_df.pivot(index='Channel', columns='Algorithm', values=metric)
    pivot = pivot.loc[target_channels]  # order channels consistently

    plt.figure(figsize=(10, 6))
    sns.heatmap(
        pivot,
        annot=True,
        fmt=".4f",  # show 4 decimal places
        cmap="YlGnBu",
        cbar_kws={'label': metric},
        annot_kws={"size": 9}
    )
    plt.title(f"{metric} - Image {target_image}")
    plt.ylabel("Channel")
    plt.xlabel("Algorithm")
    plt.tight_layout()
    plt.show()

# create single combined table
combined_table = filtered_df[['Channel', 'Algorithm'] + metric_columns]
combined_table = combined_table.sort_values(by=['Channel', 'Algorithm']).reset_index(drop=True)

# display the combined table
print("\n Combined Table for Image 1:")
print(combined_table.to_string(index=False))
